#**UNIVERSITYHACK 2022 DATATHON**:    ***Parte predictiva:***


## **INTRODUCCIÓN**

# 1. Importación de librerías y del dataset

AVISO: se trata de un código computacionarmente muy costoso que produce problemas de ejecución debido a la RAM. Aqui se presenta simplificado para una mejor comprensión de la lógica seguida, pero nosotros hemos tenido que ir lanzándolo por partes y luego agregarlas dado que de otra forma no conseguíamos ejecutarlo.

Mostramos la ejecución con una muestra para ver el procedimiento.

In [69]:
import pandas as pd 
import numpy as np 

import seaborn as sns 
import matplotlib.pyplot as plt 

from sklearn.metrics import mean_absolute_error, mean_squared_error
import math

#!pip install statsmodels==0.13.1
#!pip install pmdarima
#from pmdarima.arima import auto_arima

from array import array
from statsmodels.tsa.stattools import adfuller

import warnings 
warnings.filterwarnings('ignore')

In [70]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Necesitamos la data agrupada  por ID y  por DATE

In [71]:
df = pd.read_csv("/content/drive/MyDrive/Datathon_2022/df_ID_DATE.csv")

In [72]:
df = df.fillna(0.0)


In [73]:
df['DATE'] = pd.to_datetime(df['DATE'], errors='coerce')  

In [74]:
df

,ID,DATE,READING,DELTA
0,0,2019-02-01,7963359.0,243.0
1,0,2019-02-02,7969096.0,236.0
2,0,2019-02-03,7975263.0,335.0
3,0,2019-02-04,7982426.0,252.0
4,0,2019-02-05,7988972.0,220.0
...,...,...,...,...
888519,2748,2019-08-03,1627386.0,313.0
888520,2748,2019-09-14,1690044.0,8.0
888521,2749,2019-08-06,627196.0,0.0
888522,2749,2019-10-11,658022.0,2.0


In [75]:
print("Number of rows is = ", df.shape[0], " \nNumber of columns is = " , df.shape[1]) 

Number of rows is =  888524  
Number of columns is =  4


In [76]:
df.head(14)

,ID,DATE,READING,DELTA
0,0,2019-02-01,7963359.0,243.0
1,0,2019-02-02,7969096.0,236.0
2,0,2019-02-03,7975263.0,335.0
3,0,2019-02-04,7982426.0,252.0
4,0,2019-02-05,7988972.0,220.0
5,0,2019-02-06,7994549.0,276.0
6,0,2019-02-07,8000905.0,277.0
7,0,2019-02-08,8007305.0,193.0
8,0,2019-02-09,8011892.0,262.0
9,0,2019-02-10,8019086.0,315.0


In [77]:
df = df.rename(columns={'DATE': 'ds', 'DELTA':'y'})

In [43]:
# De esta forma lo hemos ido desgranando para conseguir las predicciones (aqui se muestran los valores conflictivos de los contadores a los que luego se hace referencia)

#prueba=df[(df['ID']>2408) & (df['ID']!=2521)& (df['ID']!=2562)& (df['ID']!=2698)]

In [81]:
#prueba2=df[(df['ID']<2726) & (df['ID']!=2521)& (df['ID']!=2562)& (df['ID']!=2698)]
prueba=df.head(40)

In [82]:
prueba['ID'] = prueba['ID'].astype(str)

In [ ]:
from fbprophet import Prophet
grouped = df.groupby('ID')
for g in grouped.groups:
    group = grouped.get_group(g)
    m = Prophet()
    m.fit(group)
    future = m.make_future_dataframe(periods=14)
    forecast = m.predict(future)
    print(forecast.tail())

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


            ds       trend  yhat_lower  yhat_upper  trend_lower  trend_upper  \
374 2020-02-10  284.171916  142.903780  418.078216   284.171914   284.171918   
375 2020-02-11  284.179909  138.670798  419.999893   284.179907   284.179912   
376 2020-02-12  284.187903  138.460840  404.684280   284.187900   284.187906   
377 2020-02-13  284.195897  146.786073  421.157156   284.195894   284.195900   
378 2020-02-14  284.203890  149.306016  407.742498   284.203887   284.203894   

     additive_terms  additive_terms_lower  additive_terms_upper     weekly  \
374       -1.257980             -1.257980             -1.257980  -1.257980   
375       -2.288253             -2.288253             -2.288253  -2.288253   
376      -16.707484            -16.707484            -16.707484 -16.707484   
377        0.713921              0.713921              0.713921   0.713921   
378       -0.968951             -0.968951             -0.968951  -0.968951   

     weekly_lower  weekly_upper  multiplicative_te

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


            ds     trend  yhat_lower  yhat_upper  trend_lower  trend_upper  \
374 2020-02-10  8.071678  -34.134670   36.254025     8.071677     8.071678   
375 2020-02-11  8.055146  -38.588968   38.301172     8.055146     8.055146   
376 2020-02-12  8.038614  -29.082464   49.647794     8.038614     8.038614   
377 2020-02-13  8.022082  -34.400455   40.803075     8.022082     8.022082   
378 2020-02-14  8.005550  -29.303187   47.190866     8.005550     8.005550   

     additive_terms  additive_terms_lower  additive_terms_upper    weekly  \
374       -6.409696             -6.409696             -6.409696 -6.409696   
375       -5.742260             -5.742260             -5.742260 -5.742260   
376        1.356866              1.356866              1.356866  1.356866   
377       -4.528224             -4.528224             -4.528224 -4.528224   
378       -2.447602             -2.447602             -2.447602 -2.447602   

     weekly_lower  weekly_upper  multiplicative_terms  \
374     -6.

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


...

            ds       trend  yhat_lower  yhat_upper  trend_lower  trend_upper  \
374 2020-02-10  150.010519   93.007861  231.715887   149.976034   150.053096   
375 2020-02-11  150.313417  101.147931  236.598368   150.270130   150.367416   
376 2020-02-12  150.616315   71.360734  214.308977   150.563101   150.681823   
377 2020-02-13  150.919213   92.612118  230.123466   150.857053   150.994077   
378 2020-02-14  151.222110   82.661187  221.929095   151.149177   151.313007   

     additive_terms  additive_terms_lower  additive_terms_upper     weekly  \
374       10.567841             10.567841             10.567841  10.567841   
375       12.920654             12.920654             12.920654  12.920654   
376       -9.016463             -9.016463             -9.016463  -9.016463   
377        8.798193              8.798193              8.798193   8.798193   
378        2.345301              2.345301              2.345301   2.345301   

     weekly_lower  weekly_upper  multiplicative_te

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


            ds       trend  yhat_lower  yhat_upper  trend_lower  trend_upper  \
374 2020-02-10  336.240811  -51.052716  671.592669   336.110308   336.388983   
375 2020-02-11  336.644018  -66.731316  652.458093   336.475111   336.828164   
376 2020-02-12  337.047225  -56.573106  674.908723   336.850162   337.269295   
377 2020-02-13  337.450432  -43.392762  711.540272   337.224233   337.701676   
378 2020-02-14  337.853639  -37.986695  722.412411   337.581353   338.151579   

     additive_terms  additive_terms_lower  additive_terms_upper     weekly  \
374      -34.797211            -34.797211            -34.797211 -34.797211   
375      -28.710428            -28.710428            -28.710428 -28.710428   
376      -16.527124            -16.527124            -16.527124 -16.527124   
377       -1.797249             -1.797249             -1.797249  -1.797249   
378       20.938619             20.938619             20.938619  20.938619   

     weekly_lower  weekly_upper  multiplicative_te

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


            ds       trend  yhat_lower  yhat_upper  trend_lower  trend_upper  \
280 2020-02-10   99.709348   15.405845  170.693523    99.705458    99.713968   
281 2020-02-11   99.865874   36.079016  197.675562    99.860978    99.871357   
282 2020-02-12  100.022400   27.275213  184.486019   100.016794   100.028743   
283 2020-02-13  100.178927    9.562291  173.980886   100.172151   100.186380   
284 2020-02-14  100.335453   22.346613  181.034750   100.327779   100.344125   

     additive_terms  additive_terms_lower  additive_terms_upper     weekly  \
280       -9.688569             -9.688569             -9.688569  -9.688569   
281       13.228591             13.228591             13.228591  13.228591   
282        6.623107              6.623107              6.623107   6.623107   
283       -8.110722             -8.110722             -8.110722  -8.110722   
284       -0.115243             -0.115243             -0.115243  -0.115243   

     weekly_lower  weekly_upper  multiplicative_te

In [ ]:
final = pd.DataFrame()
for g in grouped.groups:
    group = grouped.get_group(g)
    m = Prophet()
    m.fit(group)
    future = m.make_future_dataframe(periods=14)
    forecast = m.predict(future)    
    forecast = forecast.rename(columns={'yhat': g})
    final = pd.merge(final, forecast.set_index('ds'), how='outer', left_index=True, right_index=True)
final = final[[ g for g in grouped.groups.keys()]]
final

In [ ]:
predicciones=final.transpose()

In [20]:
#predicciones_finales['primera_semana'] = predicciones_finales.iloc[:,0:7].sum(axis=1)
#predicciones_finales['segunda_semana'] = predicciones_finales.iloc[:,8:13].sum(axis=1)

In [ ]:
predicciones

In [ ]:
predicciones = predicciones.fillna(0.0)
predicciones

In [53]:
#predicciones_finales_total = predicciones_finales.append([np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan], ignore_index=True)
#predicciones_finales_total

In [54]:
#predicciones.to_csv(r'/content/drive/MyDrive/Datathon_2022/predicciones.csv', index = False)

Por medio de este código hemos sido capaces de extraer la predicción para cada uno de los contadores para los siguientes 14 días.
Un total de 24 contadores (2407,2521,2562,2698 y los que seguían al 2726) nos han dado problemas para ejecutarlos con esta función, por lo que se han analizado individualemnte y con posterioridad se han incluido a los resultados.

Tras obtener esta predicción para los siguientes 14 días hemos sido capaces de hacer la agregacion para tener la predicción de la siguiente semana y de la siguiente.

Como explicación a este punto, primeramente para llevar a cabo el entreanmiento y la posterior aplicación del modelo ARIMA.
Sin embargo, a la hora de aplicarlo para cada uno de contadores. Es por eso por lo que pasamos a emplear el modelo de Profet.
Sin embargo, también nos han surgido problemas computacionales en lo que respecta a la RAM, por lo que ha sido un proceso costoso que,a nuestro parecer ha empeorado el rendimiento del modelo.

Nuestro principal objetivo como mejora pasa por correr correctamente y de una forma mas exhaustiva los dos modelos entrenados, tanto el prophet como el ARIMA, mejorando considerablemente las prediccciones.